# How does the natural organic matter data in the NMDC database compare across sample types?

In [ ]:
library(tidyverse)
library(jsonlite)
library(heatmaply)
library(ggExtra)

: 

This notebook demonstrates how to use the existing NMDC-runtime API endpoints (as of August 2024) to explore the natural organic matter (NOM) data present in the NMDC database. It traverses the database schema backwards, first pulling all available natural organic matter results, and then identifying the corresponding source samples. The processed data is then evaluated for quality and compared by sample type. Note that retrieving the CSV results files can be time consuming.

## Define functions to interact with API

First, we will define R functions to interact with some of the available NMDC API endpoints. Further documentation of available API endpoints is available here: https://api.microbiomedata.org/docs#/

### Define a general API call funtion to nmdc-runtime

This function provides a general-purpose way to make an API request to NMDC's runtime API. Note that this function will only return the first page of results. The function's input includes the name of the collection to access (e.g. `biosample_set`), the filter to be performed, the maximum page size, and a list of the fields to be retrieved. It returns the metadata as a dataframe.

In [ ]:
get_first_page_results <- function(collection, filter, max_page_size, fields) {
  og_url <- paste0(
      'https://api.microbiomedata.org/nmdcschema/', 
      collection, '?&filter=', filter, '&max_page_size=', max_page_size, '&projection=', fields
      )
  
  response <- jsonlite::fromJSON(URLencode(og_url, repeated = TRUE))
  
  return(response)
}

: 

### Define an nmdc-runtime API call function to include pagination

The get_next_results function uses the get_first_page_results function, defined above, to retrieve the rest of the results from a call with multiple pages. It takes the same inputs as the get_first_page_results function above: the name of the collection to be retrieved, the filter string, the maximum page size, and a list of the fields to be returned. This function returns the results as a single dataframe (can be nested). It uses the next_page_token key in each page of results to retrieve the following page.

In [ ]:
get_next_results <- function(collection, filter_text, max_page_size, fields) {
  initial_data <- get_first_page_results(collection, filter_text, max_page_size, fields)
  results_df <- initial_data$resources
  
  if (!is.null(initial_data$next_page_token)) {
    next_page_token <- initial_data$next_page_token
    
    while (TRUE) {
      url <- paste0('https://api.microbiomedata.org/nmdcschema/', collection, '?&filter=', filter_text, '&max_page_size=', max_page_size, '&page_token=', next_page_token, '&projection=', fields)
      response <- jsonlite::fromJSON(URLencode(url, repeated = TRUE))

      results_df <- results_df %>% bind_rows(response$resources)
      next_page_token <- response$next_page_token
      
      if (is.null(next_page_token)) {
        break
      }
    }
  }
  
  return(results_df)
}

: 

### Define an API request function to get object information for a given ID

This function constructs a different type of API request that takes a list of ids or similar (e.g. `biosample` ids as retrieved above). The `id_field` input is a string of the name of the id field name (e.g. `id` or `has_output`), the name of the new collection to be queried, the name of the field to match the previous ids on in the new collection, and a list of the fields to be returned.

In [ ]:
get_results_by_id <- function(collection, match_id_field, id_list, fields, max_id = 50) {
    # collection: the name of the collection to query
    # match_id_field: the field in the new collection to match to the id_list
    # id_list: a list of ids to filter on
    # fields: a list of fields to return
    # max_id: the maximum number of ids to include in a single query
    
    # If id_list is longer than max_id, split it into chunks of max_id
    if (length(id_list) > max_id) {
        id_list <- split(id_list, ceiling(seq_along(id_list)/max_id))
    } else {
        id_list <- list(id_list)
    }
    
    output <- list()
    for (i in 1:length(id_list)) {
        # Cast as a character vector and add double quotes around each ID
        mongo_id_string <- as.character(id_list[[i]]) %>%
            paste0('"', ., '"') %>%
            paste(collapse = ', ')
        
        # Create the filter string
        filter = paste0('{"', match_id_field, '": {"$in": [', mongo_id_string, ']}}')
        
        # Get the data
        output[[i]] = get_next_results(
            collection = collection,
            filter = filter,
            max_page_size = max_id*3, #assumes that there are no more than 3 records per query
            fields = fields
        )
    }
    output_df <- bind_rows(output)
}

: 

### Define an API request function to identify the database collection for an ID

The functions above rely on knowing the MongoDB "collection" to search for relevant objects. But what if you don't know what the collection is called? This function can be used to obtain the the names of the collection that an object is part of.

In [ ]:
get_collection_by_id <- function(id) {
  
  # Create API endpoint URL
  url <- paste0("https://api.microbiomedata.org/nmdcschema/ids/", id, "/collection-name")

  # Retrieve the JSON result from the API endpoint URL
  response <- jsonlite::fromJSON(URLencode(url, repeated = TRUE))
  
  # Extract the collection name from the response
  return(response$collection_name)
}

: 

## Query NMDC API

Now we will use the functions defined above to retrieve data from the NMDC database. In each step, we will 

### Get all data objects with NOM results

We will start by using the `get_next_results` function to retrieve all available processed data records containing NOM results. We are querying the `data_object_set` collection for objects that have the type `FT ICR-MS Analysis Results`. 

In [ ]:
nom_dobj_df <- get_next_results(
    collection = "data_object_set", 
    filter_text = '{"data_object_type": {"$regex": "FT ICR-MS Analysis Results"}}', 
    max_page_size = 100, 
    fields = "id,name,url,was_generated_by,md5_checksum"
    )

: 

### Example of identifying a collection

We will use the `get_collection_by_id` function defined above to identify the name of the next collection to search in. Here, we have a list of IDs from the `data_object_set` collection, and we need to know the name of the collection containing the data analysis workflow information from the previous step.

In [ ]:
# What collection holds the analysis process that produced our processed data object?
get_collection_by_id(nom_dobj_df$was_generated_by[1])

: 

### Get all NOM analysis results

Now that we have a dataframe of data object information, we can use the data object IDs to identify the analysis processes that produced our results. In this case, we are looking in the `nom_analysis_activity_set` for any records whose output is listed as one of our data object IDs.

In [ ]:
nom_analysis_df <- get_results_by_id(
    collection = "nom_analysis_activity_set",
    match_id_field = "has_output",
    id_list = nom_dobj_df$id,
    fields = "id,has_input,has_output",
    max_id = 20
) %>%
  mutate(has_input = unlist(has_input),
         has_output = unlist(has_output))

: 

### Get all data objects containing raw NOM data

Continuing to work backwards towards the original biosample, we now use the analysis object IDs to identify the raw data objects that were analyzed. This step is also looking in the collection `data_object_set` that was used above. The difference is that this time, rather than filtering by type, we are looking for data objects that match our list of data object IDs that were input to the NOM analysis processes.

In [ ]:
nom_raw_df <- get_results_by_id(
    collection = "data_object_set",
    match_id_field = "id",
    id_list = nom_analysis_df$has_input,
    fields = "id,was_generated_by,name,md5_checksum",
    max_id = 20
)

: 

### Get all sample processing records that produced NOM data

Using the list of raw data object IDs obtained above, we now search the `omics_processing_set` collection for records of sample processing that produced raw NOM data. Here we are looking for sample processing records whose output field matches the list of raw data object IDs.

In [ ]:
nom_omprc_df <- get_results_by_id(
    collection = "omics_processing_set",
    match_id_field = "has_output",
    id_list = nom_raw_df$id,
    fields = "id,has_input,has_output",
    max_id = 20
) %>%
  mutate(has_input = unlist(has_input),
         has_output = unlist(has_output))

: 

### Get all records for biosamples that were processed for NOM analysis

Finally, we are able to use the inputs to the sample processing records obtained above to identify the original biosamples used. This will allow us to connect the sampling metadata to the processed NOM results. Here, we can add fields to the API query to retrieve more information about the original biosample. 

In this case, we are interested in the environments from which the samples were taken, so the `fields` argument includes the trio of environmental terms required for each NMDC biosample: `env_broad_scale`, `env_local_scale`, and `env_medium`. 

Passing an empty string to the `fields` argument of the API functions defined above returns all fields for the class being queried, which can be useful but slow. More information about the metadata fields for the `Biosample` class is available here: https://microbiomedata.github.io/nmdc-schema/Biosample/

In [ ]:
nom_biosample_df <- get_results_by_id(
    collection = "biosample_set",
    match_id_field = "id",
    id_list = nom_omprc_df$has_input,
    fields = "id,name,part_of,env_broad_scale,env_local_scale,env_medium",
    #fields = "",
    max_id = 20
) %>%
  # Remove duplicate rows (some biosamples produce more than one processed sample)
  distinct()

: 

### Join all results dataframes

Now that we have covered each step that connects a biosample to its processed data output, we can join the dataframes by their common ID columns to produce one dataframe.

In [ ]:
# Starting from the biosample dataframe...
nom_objects_joined <- nom_biosample_df %>%
  
  # Rename biosample columns with the prefix "bsm" for clarity
  rename_with(~ paste0("bsm.", .x)) %>%
  
  # Rename sample processing columns with the prefix "omprc"
  # Join records where the biosample ID is the input to the sample processing
  full_join(rename_with(nom_omprc_df, ~ paste0("omprc.", .x)), 
            by = join_by(bsm.id == omprc.has_input),
            keep = TRUE) %>%
  
  # Rename raw data object columns with the prefix "raw"
  # Join records where the raw data ID is the output of the sample processing
  full_join(rename_with(nom_raw_df, ~ paste0("raw.", .x)), 
            by = join_by(omprc.has_output == raw.id),
            keep = TRUE) %>%
  
  # Rename analysis columns with the prefix "analysis"
  # Join records where the raw data ID is the input of the analysis process
  full_join(rename_with(nom_analysis_df, ~ paste0("analysis.", .x)), 
            by = join_by(raw.id == analysis.has_input),
            keep = TRUE) %>%
  
  # Rename processed data object columns with the prefix "dobj"
  # Join records where the processed data ID is the output of the analysis process
  full_join(rename_with(nom_dobj_df, ~ paste0("dobj.", .x)), 
            by = join_by(analysis.has_output== dobj.id),
            keep = TRUE) %>%
  
  # Clean up: remove duplicate rows produced by joins
  distinct() %>%
  
  # Flatten list-columns
  # List-columns appear when a field included in the query is not flat (e.g. https://microbiomedata.github.io/nmdc-schema/env_medium/)
  unnest(where(is.data.frame), names_sep = ".", keep_empty = TRUE) %>%
  unnest(where(is.data.frame), names_sep = ".", keep_empty = TRUE) %>%
  mutate(across(where(is.list), as.character)) %>%
  
  # Remove duplicate processed data objects
  filter(!duplicated(dobj.md5_checksum))

: 

## Label sample types

In this notebook we want to explore how natural organic matter varies across sample types. We can label the sample types using the environmental metadata terms included in the biosample query earlier.

Environmental metadata terms in the NMDC database are taken from controlled vocabularies, predominantly the Environment Ontology (ENVO). ENVO terms have a term name as well as an ID number (e.g. `peat soil [ENVO:00005774]`). Additional information about ENVO terms and their relationships can be found here: https://www.ebi.ac.uk/ols4/ontologies/envo

Here we use the `env_medium` field to identify whether our NOM samples came from soil, sediment, or water. `env_medium` actually contains a multilevel structure to represent the environmental term. `env_medium.has_raw_value` holds a string combining the term name and ID, `env_medium.term.id` contains the term ID number, and `env_medium.term.name` contains the term name, as shown below: 

In [ ]:
nom_objects_joined$bsm.env_medium.has_raw_value[1]

: 

In [ ]:
nom_objects_joined$bsm.env_medium.term.id[1]

: 

In [ ]:
nom_objects_joined$bsm.env_medium.term.name[1]

: 

Note that all of these fields should be filled out for all samples, however, due to differences in sample metadata submission by users, there may be gaps. Therefore we check multiple fields here to ensure that all samples are labeled.

In [ ]:
nom_objects_joined_labeled <- nom_objects_joined %>%

  mutate(bsm.sample_type_manual = case_when(
    bsm.env_medium.term.id %in% c("ENVO:00001998", "ENVO:00002259", "ENVO:00002261", 
                                  "ENVO:00005750", "ENVO:00005760", "ENVO:00005761", 
                                  "ENVO:00005773", "ENVO:00005774", "ENVO:00005802", 
                                  "ENVO:01001616") ~ "soil",
    
    bsm.env_medium.term.id %in% c("ENVO:00002007") ~ "sediment",
    bsm.env_medium.term.id %in% c("ENVO:01000017") ~ "sand",
    bsm.env_medium.term.id %in% c("ENVO:00002042") ~ "water",
    
    str_detect(bsm.env_medium.term.name, "soil") ~ "soil",
    str_detect(bsm.env_medium.has_raw_value, "soil") ~ "soil",
    str_detect(bsm.env_medium.term.name, "water") ~ "water"
    ))

# Cleanup
rm(nom_dobj_df, nom_analysis_df, nom_raw_df, nom_omprc_df, nom_biosample_df, nom_objects_joined)

: 

## Pull and process results files

The processed data objects pulled from the API above contain references to the results files, not the results themselves. We can use the `url` field of the data object class to retrieve the stored data.

The NMDC natural organic matter workflow produces a results data table saved as a CSV. We will read the CSV file into R from each data object URL. 

Note that this step is the most time-consuming; it may take up to 40 minutes.

In [ ]:
processed_data_csvs <- data.frame(data_object_url = nom_objects_joined_labeled$dobj.url) %>%
  filter(!is.na(data_object_url)) %>%
  mutate(csv_nested = lapply(data_object_url, read.csv))

: 

The results table contains a lot of information. In this notebook, we are going to use the confidence score, molecular formula assignment, hydrogen/carbon ratio, and oxygen/carbon ratio columns.

In [ ]:
colnames(processed_data_csvs$csv_nested[[1]])

: 

### Select best assignments

Rarely, a peak may match more than one molecular formula with high confidence. In these cases there would be more than one row for the peak, with duplicate indices and m/z values but different molecular formulae. In this case we will select the row with the highest confidence score to ensure that peaks are not duplicated within a results file.

In [ ]:
processed_data_csvs <- processed_data_csvs %>%
  # For each results dataframe
  mutate(csv_nested = lapply(csv_nested, 
    function(x) {
      x %>%
        # Keep the row with the highest confidence score for every m/z value
        arrange(-Confidence.Score) %>%
        distinct(m.z, .keep_all = TRUE)
      }
    )
  )

: 

### Quality metrics for processed data

We can extract the columns we need from the results table and calculate some quality metrics.

Percent Assigned Peaks: The percentage of peaks that were given a molecular formula assignment.

Confidence Score: The NMDC NOM workflow is based off of the CoreMS package, which outputs a confidence score for each molecular formula assignment. This allows us to potentially filter out less reliable assignments and move forward with higher confidence molecular formula identifications.

In [ ]:
# Extract relevant columns and calculate some statistics for the processed data
processed_data_qc <- processed_data_csvs %>%
  
  mutate(unassigned_count   = vapply(csv_nested, function(x) sum((x$Molecular.Formula == "")), 
                                     FUN.VALUE = 1, USE.NAMES = FALSE),
         
         assigned_count     = vapply(csv_nested, function(x) sum((x$Molecular.Formula != "")), 
                                     FUN.VALUE = 1, USE.NAMES = FALSE),
         
         assigned_perc      = assigned_count / (unassigned_count + assigned_count),
         
         confidence_scores  = lapply(csv_nested, function(x) x$Confidence.Score))

: 

In [ ]:
ggplot(processed_data_qc) +
  geom_histogram(aes(x = assigned_perc)) +
  ggtitle("Percent Assigned Peaks for Processed Data Files") +
  xlab("Percent of peaks with assigned formula") +
  ylab("Number of processed data files")

: 

In [ ]:
data.frame(All_Confidence_Scores = unlist(processed_data_qc$confidence_scores)) %>%
  ggplot() +
    geom_histogram(aes(x = All_Confidence_Scores), bins = 30) +
    ggtitle("Confidence Scores for All Peak/Formula Assignments") +
    xlab("Confidence score") +
    ylab("Number of peaks")

: 

We can use the quality metrics to filter our data before moving forward with analysis.

First, we will remove samples with less than 30% of their peaks assigned. Then we apply a filter to each remaining data file to remove molecular formula assignments with confidence scores lower than 0.75.

In [ ]:
# Filter out samples with low percent assigned
processed_data_qc <- processed_data_qc %>%
  filter(assigned_perc > 0.3) %>%
  
  # Within data, filter out rows with low confidence score
  mutate(csv_nested = lapply(csv_nested, function(x) filter(x, Confidence.Score > 0.75))) %>%

  # Now that filtering is done, extract the rest of the data we want
  mutate(molecular_formulae = lapply(csv_nested, function(x) x$Molecular.Formula),
         HC                 = lapply(csv_nested, function(x) x$H.C),
         OC                 = lapply(csv_nested, function(x) x$O.C),
         # re-extract filtered confidence scores
         confidence_scores  = lapply(csv_nested, function(x) x$Confidence.Score)
  ) %>%
  
  # Remove nested dataset column for speed and memory
  select(-csv_nested)

: 

## Visualizing natural organic matter by sample type

First we will see how many processed data objects from each sample type remain after quality filtering.

In [ ]:
processed_data_qc %>%
  left_join(select(nom_objects_joined_labeled, dobj.url, bsm.sample_type_manual),
              by = join_by(data_object_url == dobj.url)) %>%
  pull(bsm.sample_type_manual) %>%
  table()

: 

Most of the remaining data comes from soil samples. Water has the fewest results remaining. We will use 33 of each type moving forward to visualize all sample types.

### Molecular formula heatmap

One way to compare samples is to investigate what molecular formulae they have in common. We will visualize this by creating a heatmap with a subset of samples of each type.

In [ ]:
set.seed(10)

# Starting from the QC filtered data
samples_for_heatmap <- processed_data_qc %>%
  
  # Add biosample information
  left_join(select(nom_objects_joined_labeled, 
                   dobj.id, dobj.url, bsm.sample_type_manual), #, bsm.part_of, 
                   #bsm.lat_lon.latitude, bsm.lat_lon.longitude), 
            by = join_by(data_object_url == dobj.url)) %>%
  
  # Randomly sample 33 of each sample type (limited by only 33 water samples)
  group_by(bsm.sample_type_manual) %>%
  slice_sample(n = 33) %>%
  ungroup()

: 

We also filter down to molecular formulae that appear in at least 5 samples, both to make the plots less computationally demanding and to remove formulae that are less useful for comparing samples.

In [ ]:
filtered_formulae <- samples_for_heatmap$molecular_formulae %>%
  # Get unique list of formulae from each results table
  lapply(unique) %>%
  unlist() %>%
  # Count occurrences of each formula
  table() %>%
  as.data.frame() %>%
  setNames(c("Formula", "Freq")) %>%
  # Remove stray empty strings and filter by frequency
  filter(Formula != "") %>%
  filter(Freq >= 5) %>%
  mutate(Formula = as.character(Formula)) %>%
  # Pull character vector of formulae appearing at least 5x in the random subsample
  pull(Formula)

: 

One way to visualize similarities between samples is to investigate their shared formulae. We will construct a presence-absence heatmap showing which formulae are present in which results.

In [ ]:
# Initialize presence absence matrix
# Rows are processed data (results data tables)
# Columns are molecular formulae

holder <- matrix(, nrow = nrow(samples_for_heatmap), ncol = length(filtered_formulae))

for (i in 1:nrow(samples_for_heatmap)) {
  holder[i,] <- ifelse(filtered_formulae %in% samples_for_heatmap$molecular_formulae[[i]], 
                       1, 0)
}

rownames(holder) <- samples_for_heatmap$data_object_url
colnames(holder) <- filtered_formulae

: 

We will also construct a colored sidebar indicating which sample type each results row comes from. This will help visualize whether or not sample types cluster together.

In [ ]:
# Set up color vector for side grouping column
sample_type_color_vec <- case_when(
  samples_for_heatmap$bsm.sample_type_manual == "soil" ~ "saddlebrown",
  samples_for_heatmap$bsm.sample_type_manual == "sediment" ~ "slategray",
  samples_for_heatmap$bsm.sample_type_manual == "water" ~ "deepskyblue",
  samples_for_heatmap$bsm.sample_type_manual == "sand" ~ "goldenrod")

: 

The heatmap below shows molecular formulae that are present (orange) or absent (blue) in each processed data record. On the left, the dendrogram shows how the records clustered together based on similarity of molecular formulae. The column on the left shows sample type: blue is water, gray is sediment, yellow is sand, and brown is soil.

In [ ]:
heatmap(holder, Colv = NA, RowSideColors = sample_type_color_vec, 
        labRow = FALSE, labCol = FALSE, scale = "none",
        main = "NOM results clustered by similarity in molecular formulae",
        xlab = "Molecular formulae",
        ylab = "Samples", 
        keep.dendro = FALSE, 
        col = c("darkblue", "orange"))

: 

### Van Krevelen plots

Van Krevelen plots are a common way to visualize samples of organic material. Each point on the scatterplot represents a molecular formula. The x-axis represents the hydrogen/carbon ratio of the molecule, and the y-axis represents the oxygen/carbon ratio. In this example, the data has been grouped by sample type rather than plotting all results individually.

In [ ]:
# Using the same subset of samples chosen for the heatmap
van_krevelen_data <- samples_for_heatmap %>%
  select(bsm.sample_type_manual, HC, OC, molecular_formulae) %>%
  unnest() %>% 
  # Filter down to formulae that appear at least 5 times (see above)
  filter(molecular_formulae %in% filtered_formulae) %>% 
  distinct() 

ggplot(van_krevelen_data, aes(x = OC, y = HC, color = bsm.sample_type_manual)) +
  geom_point(alpha = 0.5, size = 1) +
  facet_wrap(~bsm.sample_type_manual, nrow = 1) +
  theme(legend.position = "none") +
  coord_fixed() +
  scale_color_manual("legend", values = c("soil" = "saddlebrown",
                                          "sediment" = "slategray",
                                          "water" = "deepskyblue",
                                          "sand" = "goldenrod")) +
  ggtitle("Van Krevelen Plots by Sample Type") +
  xlab("Oxygen/Carbon Ratio") +
  ylab("Hydrogen/Carbon Ratio")

: 

The water samples appear to have the most identified molecular formulae in the results. This may be because it is easier to extract NOM from water in lab processing. To visualize shared formulae between samples, we can plot all the sample types together.

In [ ]:
van_krevelen_data %>%
  mutate(exists = TRUE) %>%
  pivot_wider(names_from = bsm.sample_type_manual, values_from = exists) %>%
  mutate(across(where(is.logical), ~ !is.na(.x))) %>%
  
  # Label all shared formulae
  mutate(in_soil_only = soil & !sediment & !water & !sand,
         in_water_only = !soil & !sediment & water & !sand,
         in_sediment_only = !soil & sediment & !water & !sand,
         in_sand_only = !soil & !sediment & !water & sand,
         in_all_sample_types = sediment & soil & water) %>%
  
  # Back to long format for ggplot
  select(-soil, sediment, water, sand) %>%
  pivot_longer(cols = starts_with("in_"), 
               names_to = "sample_type", values_to  = "presence") %>%
  filter(presence) %>%
  
  # Make nicer labels for plot
  mutate(sample_type = case_when(
    sample_type == "in_soil_only" ~ "Unique to soil",
    sample_type == "in_water_only" ~ "Unique to water",
    sample_type == "in_sediment_only" ~ "Unique to sediment",
    sample_type == "in_sand_only" ~ "Unique to sand",
    sample_type == "in_all_sample_types" ~ "Present in all sample types"
  )) %>%
  
  # Construct scatterplot
  ggplot(aes(x = OC, y = HC, color = sample_type)) +
    geom_point(alpha = 0.7, size = 1, shape = 1) +
    theme(legend.position = "bottom",
          plot.title = element_text(margin = margin(t = 1, unit = "cm"), size = 12)) +
    labs(title = "Van Krevelen: unique and shared molecular formulae by sample type",
         x = "Oxygen/Carbon Ratio", y = "Hydrogen/Carbon Ratio", color = "Sample Type") +
    guides(color = guide_legend(override.aes = list(alpha = 1, size = 3))) +
    scale_color_manual("legend", 
                       values = c("Unique to soil" = "saddlebrown", 
                                 "Unique to water" = "deepskyblue", 
                                 "Unique to sediment" = "slategray", 
                                 "Unique to sand" = "goldenrod",
                                 "Present in all sample types" = "red")) -> shared_van_krevelen

# Add marginal density plots
ggMarginal(shared_van_krevelen, groupColor = TRUE, groupFill = TRUE)

: 